# Widgets

In [ ]:
import moldesign as mdt
import nbmolviz

In [ ]:
small_molecule = mdt.from_name('butadiene')
protein = mdt.from_pdb('3aid')

## Small molecule visualizer

Clicking on an atom in 2D or 3D will select the atoms in *both* 2D and 3D. It will also print out information about the atom in a widget below the viewers.

Constructor: `moldesign.molecules.atomcollections.AtomContainer.draw()`

Class definition: `moldesign.display.selector.SelectionGroup`

In [ ]:
small_molecule.draw()

## Large molecule visualizer

Same as small molecule visualizer, but doesn't display a 2D representation.

Constructor: `moldesign.molecules.atomcollections.AtomContainer.draw()`

Class definition: `moldesign.display.selector.SelectionGroup`

In [ ]:
viewer = protein.draw()
viewer

In [ ]:
viewer.sphere()

## Symmetrizer

Visually depicts "symmetry groups" like mirror planes and rotational symmetries.

It shows exact symmetries and approximate ones. Users can impose an exact symmetry with the "Symmetrize" button.

When first called, and whenever the molecular position changes due to a "Symmetrize" operation, we call a docker container called "symmol" to actually calculate the symmetries.

Class definition: `moldesign.widgets.symmetry.Symmetrizer`

In [ ]:
mdt.widgets.Symmetrizer(small_molecule)

## ResidueSelector

This widget allows users to build a selection of atoms and/or residues visually, then pull the results into their python script.

Class definition: `moldesign.widgets.selection.ResidueSelector`

In [ ]:
residue_selector = mdt.widgets.ResidueSelector(protein)
residue_selector

In [ ]:
residue_selector.selected_atoms

## Bond Selector

This allows users to build selections of atoms, bonds or both, usually with small molecules. It's similar to ResidueSelector, but aimed at smaller molecules.

Out of the box, 3DMol.js doesn't give us much control over bonds. This widget is built on a hacked Viewer class - `moldesign.viewer.bondclicker.BondClicker` - that allows us to set custom bond colors and register callbacks from clicks on bonds.

Class definition: `moldesign.widgets.selection.BondSelector`

In [ ]:
bondselector = mdt.widgets.BondSelector(small_molecule)
bondselector

The currently selected atoms and bonds here can be accessed in python code:

In [ ]:
print(bondselector.selected_bonds)

In [ ]:
bondselector.selected_atoms

## Geometry Builder

This widget also uses the `BondClicker` class to allow us to select bonds. `BondClicker` is very slow to redraw, however, so this widget ends up feeling pretty clunky.

The UI here is also pretty bad - selecting a dihedral angle to change requires selecting 4 atoms or 3 bonds, which tends to get confusing fast.

Class definition: `moldesign.widgets.geombuilder.GeometryBuilder`

In [ ]:
mdt.widgets.GeometryBuilder(small_molecule)

## Orbital visualizer

The orbitals need to be calculated before they can be visualized - we'll do that here:

Class definition: `moldesign.widgets.orbgitals.OrbitalViewer`

Usually constructed using `moldesign.molecules.molecule.MolDrawingMixin.draw_orbitals()`

In [ ]:
small_molecule.set_energy_model(mdt.models.RHF, basis='sto-3g')
small_molecule.calculate()

In [ ]:
small_molecule.draw_orbitals()

To draw the orbitals, python generates a "Gaussian Cube File" containing 3D information that it passes to JavaScript (this can be slow, especially it needs to run in a Docker container). To draw the orbitals, we pass the cube file to the `glviewer.AddIsoSurface` method.

### Trajectory Viewer

Trajectory animations are drawn using the  class, but users will usually draw one by calling `trajectory.draw()`.

Here, we'll create a trajectory by messing with our small molecule's internal geometry. This will take a second because we'll calculate the electronic wavefunction every step of the way.

Class definition: `moldesign.widgets.trajectory.TrajectoryViewer`

Usually constructed with: `moldesign.molecules.trajectory.Trajectory.draw()`

In [ ]:
import numpy as np

traj = mdt.Trajectory(small_molecule)
for i in np.arange(1.0, 2.0, 0.1):
    mdt.set_distance(small_molecule.atoms[1], small_molecule.atoms[2], i*mdt.units.angstrom)
    small_molecule.calculate()
    traj.new_frame()

Note that the "Play" button will actually block until the animation completes. The new version of `ipywidgets` is rumored to have a "play/pause" widget that can fix this.

In [ ]:
traj.draw()

### Trajectory Orbital Viewer

This is like a trajectory viewer but also lets users see orbital animations. The implementation is pretty hacky.

Class definition: `moldesign.widgets.trajectory.TrajectoryOrbViewer`

Usually constructed with: `moldesign.molecules.trajectory.Trajectory.draw_orbitals()`

In [ ]:
traj.draw_orbitals()

### Forcefield assignment viewer

This allows users to visualize protein structure problems that prevent a simulation from being created.

It's essentially a smart viewer for logging output - to visualize it, click on the `ERRORS/WARNINGS` tab.

Class definition: `moldesign.widgets.parameterization.ParameterizationDisplay`

Usually constructed in: `moldesign.interfaces.ambertools.run_tleap()`

In [ ]:
mdt.assign_forcefield(protein)